In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import Preprocessing as prep
import Visualization as vis


import warnings
warnings.filterwarnings("ignore")               # 경고 메시지 무시
# warnings.filterwarnings(action='default')     # 경고 메시지 활성화

plt.rcParams['axes.unicode_minus'] = False      # Glyph 8722 오류 해결을 위한 설정
plt.rcParams["figure.figsize"] = (12, 9)        # 그림 크기 설정
plt.rc('font', family="Malgun Gothic")       # 한글 폰트 설정
sns.set_theme(palette="Pastel1")                                 # Seaborn의 테마 자동 설정